<a href="https://colab.research.google.com/github/Dedo488/Website/blob/master/template_miniproyecto_1_pig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini-Tarea 1: Análisis de datos de preferencias musicales

La presente actividad debe ser realizada de forma individual. El formato de entregar es el archivo .ipynb con todas las celdas ejecutadas. Las secciones donde se planteen preguntas de forma explícita, deben ser respondida en celdas de texto, y no se aceptará solo el output de una celda de código como respuesta.

**Fecha de entrega:** martes 13 de octubre de 2020, 09:00 hrs.

**Nombre alumno:** Diego Pablo Figueroa Ramírez

# Apache Pig

## Instalación de ambiente

En esta sección subimos hadoop, executamos el instalador y descargamos la información para el miniproyecto.

In [1]:
#Subir hadoop_colab_installer.py
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving hadoop_colab_installer.py to hadoop_colab_installer.py
User uploaded file "hadoop_colab_installer.py" with length 13493 bytes


In [2]:
exec(open('hadoop_colab_installer.py').read())

Active services:
2165 JobHistoryServer
2086 NodeManager
1782 DataNode
2204 Jps
1629 ResourceManager
1711 NameNode



## Descarga de dataset

In [3]:
!wget -q --show-progress http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz
!tar xzf lastfm-dataset-1K.tar.gz

lastfm-dataset-1K.t 100%[===================>] 641.58M  1.27MB/s    in 9m 41s  


## Actividad 0

En esta sección, creamos los directorios userid-profile y userid-otros en HDFS. 

Después, insertamos a través del comando -put los archivos en los directorio, quedando así que la informacióne está en HDFS.

Posteriormente, creamos un UDF en Jython "my_pig_udfs2.py" que está basada en el ejemplo de clases, solamente que un poco más reducida. 

In [4]:
!hdfs dfs -mkdir userid-profile
!hdfs dfs -mkdir userid-otros
!hdfs dfs -put lastfm-dataset-1K/userid-profile.tsv userid-profile
!hdfs dfs -put lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv userid-otros

In [5]:
%%writefile my_pig_udfs2.py

from pig_util import outputSchema
import re
import array

@outputSchema('detected:boolean')
def empty_or_null_detector( gender ):
  if gender is None:
    return True
  else:
    return False  
  gender = gender.tostring() # 'array' to 'str'
  if len( gender ) == 0:
    return True
  else: 
    return False

Writing my_pig_udfs2.py


## Actividad 1

En esta actividad escribimos la file "actividad1.pig" en la cual a través del lenguaje Pig Latin realizamos las distintas acciones para responder la pregunta. 

La estructura es: Load -> Transformar -> Store. Donde en la etapa de transformación, agrupamos, realizamos un count, ordenamos y restringimos.

El resultado obtenido es: 

artist_name conteo

Radiohead	115209

The Beatles	100338

Nine Inch Nails	84421

Muse	63351

Coldplay	62251

Depeche Mode	59910

Pink Floyd	58561

Death Cab For Cutie	58083

Placebo	53543

Elliott Smith	50278

Por último, el artista más popular es Radiohead con 115209 números de reproducciones de sus canciones y el último del top-ten es Elliott Smith con 50278. 

In [6]:
%%writefile actividad1.pig

raw = LOAD 'userid-otros' USING PigStorage('\t') AS (userid:chararray, timeztamp:chararray, artist_id:chararray, artist_name:chararray, track_id:chararray, track_name:chararray);

agrupar = GROUP raw BY artist_name;

contar = FOREACH agrupar GENERATE $0 as artist_name, COUNT($1) AS conteo;

ordenar = ORDER contar BY conteo DESC;

restringir = LIMIT ordenar 10;

STORE restringir INTO 'actividad1' USING PigStorage( '\t' );

Writing actividad1.pig


In [7]:
!pig -f actividad1.pig

20/10/11 20:56:15 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
20/10/11 20:56:15 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
20/10/11 20:56:15 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2020-10-11 20:56:15,377 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2020-10-11 20:56:15,377 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1602449775373.log
2020-10-11 20:56:15,907 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2020-10-11 20:56:15,981 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-10-11 20:56:15,981 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://localhost:9000
2020-10-11 20:56:16,682 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session:

In [8]:
!hdfs dfs -ls

Found 3 items
drwxr-xr-x   - root supergroup          0 2020-10-11 21:02 actividad1
drwxr-xr-x   - root supergroup          0 2020-10-11 20:56 userid-otros
drwxr-xr-x   - root supergroup          0 2020-10-11 20:55 userid-profile


In [9]:
!hdfs dfs -cat actividad1/part-r-00000

Radiohead	115209
The Beatles	100338
Nine Inch Nails	84421
Muse	63351
Coldplay	62251
Depeche Mode	59910
Pink Floyd	58561
Death Cab For Cutie	58083
Placebo	53543
Elliott Smith	50278


Responda la siguiente pregunta:

¿ Cuál es el artista más popular y el menos popular del ranking top-10 ?


## Actividad 2

En esta actividad, utilizamos el lenguaje Pig Latin y la UDF: 'my_pig_udfs2' para responder la pregunta. 

El archivo "actividad2.pig" tiene la misma estructura que la anterior, la diferencia está en que se utiliza la UDF para filtrar los null de la variable género. Cabe aludir que funciona de igual manera con FILTER userid_profile BY gender IS NOT NULL. 

El resultado es el siguiente: 

gender conteo

m	63784

f	43748

Por lo tanto, la distribución de género nos indica que más hombres que mujeres escucharon al artista Radiohead. Para los hombres son 63784 número de reproducciones y para las mujeres son 43748. 


In [10]:
%%writefile actividad2.pig

REGISTER 'my_pig_udfs2.py' USING jython AS my_pig_udfs2;

userid_profile = LOAD 'userid-profile' USING PigStorage('\t') AS (userid_p:chararray, gender:chararray, age:long, country:chararray, signup:chararray);

userid_otros = LOAD 'userid-otros' USING PigStorage('\t') AS (userid_o:chararray, timeztamp:chararray, artist_id:chararray, artist_name:chararray, track_id:chararray, track_name:chararray);

nullgenero = FILTER userid_profile BY my_pig_udfs2.empty_or_null_detector(gender) == FALSE;

juntar_tablas = JOIN nullgenero BY userid_p, userid_otros BY userid_o;

alias = FILTER juntar_tablas BY (artist_name == 'Radiohead');

agrupar = GROUP alias BY gender;

contar = FOREACH agrupar GENERATE $0 as gender, COUNT($1) AS conteo;

ordenar = ORDER contar BY conteo DESC;

restringir = LIMIT ordenar 10;

STORE restringir INTO 'actividad2' USING PigStorage( '\t' );

Writing actividad2.pig


In [11]:
!pig -f actividad2.pig

20/10/11 21:02:15 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
20/10/11 21:02:15 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
20/10/11 21:02:15 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2020-10-11 21:02:15,971 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2020-10-11 21:02:15,972 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1602450135970.log
2020-10-11 21:02:16,598 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2020-10-11 21:02:16,677 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-10-11 21:02:16,677 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://localhost:9000
2020-10-11 21:02:17,188 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session:

In [12]:
!hdfs dfs -ls

Found 4 items
drwxr-xr-x   - root supergroup          0 2020-10-11 21:02 actividad1
drwxr-xr-x   - root supergroup          0 2020-10-11 21:11 actividad2
drwxr-xr-x   - root supergroup          0 2020-10-11 20:56 userid-otros
drwxr-xr-x   - root supergroup          0 2020-10-11 20:55 userid-profile


In [13]:
!hdfs dfs -cat actividad2/part-r-00000

m	63784
f	43748


Responda la siguiente pregunta:

¿ Cuál es la cantidad de hombres y mujeres que han escuchado alguna canción del artista más popular ?

## Actividad 3

En esta actividad, utilizamos el lenguaje Pig Latin y la UDF: 'my_pig_udfs2' para responder la pregunta. 

El archivo "actividad2.pig" tiene la misma estructura que la anterior, la diferencia está en que se utiliza la UDF para filtrar los null de la variable edad. No se ocupa el LIMIT 10, con el objetivo de tener toda la distribución de edad y se ordena por edad y no por conteo.

El resultado es el siguiente: 

age conteo

3	2

4	14

7	2

15	2

16	122

17	444

18	65

19	4514

20	5760

21	1589

22	7038

23	2552

24	1313

25	1289

26	2527

27	2360

28	1330

29	359

30	617

31	146

32	433

33	1441

34	404

35	1543

36	1962

38	422

39	213

42	13

48	35

75	109

103	1

Por lo tanto, la distribución de edad nos indica que los usuarios varían desde 3 a 103 años y 1543 personas de 35 años han escuchado al artista más popular Radiohead. También podemos notar que este artista es más popular entre las edades [19 a 22]. 

In [14]:
%%writefile actividad3.pig

REGISTER 'my_pig_udfs2.py' USING jython AS my_pig_udfs2;

userid_profile = LOAD 'userid-profile' USING PigStorage('\t') AS (userid_p:chararray, gender:chararray, age:long, country:chararray, signup:chararray);

userid_otros = LOAD 'userid-otros' USING PigStorage('\t') AS (userid_o:chararray, timeztamp:chararray, artist_id:chararray, artist_name:chararray, track_id:chararray, track_name:chararray);

nulledad = FILTER userid_profile BY my_pig_udfs2.empty_or_null_detector(age) == FALSE;

juntar_tablas = JOIN nulledad BY userid_p, userid_otros BY userid_o;

alias = FILTER juntar_tablas BY (artist_name == 'Radiohead');

agrupar = GROUP alias BY age;

contar = FOREACH agrupar GENERATE $0 as age, COUNT($1) AS conteo;

ordenar = ORDER contar BY $0 ASC;

STORE ordenar INTO 'actividad3' USING PigStorage( '\t' );

Writing actividad3.pig


In [15]:
!pig -f actividad3.pig

20/10/11 21:11:16 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
20/10/11 21:11:16 INFO pig.ExecTypeProvider: Trying ExecType : MAPREDUCE
20/10/11 21:11:16 INFO pig.ExecTypeProvider: Picked MAPREDUCE as the ExecType
2020-10-11 21:11:16,823 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2020-10-11 21:11:16,823 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1602450676820.log
2020-10-11 21:11:17,418 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2020-10-11 21:11:17,488 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-10-11 21:11:17,488 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: hdfs://localhost:9000
2020-10-11 21:11:18,020 [main] INFO  org.apache.pig.PigServer - Pig Script ID for the session:

In [16]:
!hdfs dfs -ls

Found 5 items
drwxr-xr-x   - root supergroup          0 2020-10-11 21:02 actividad1
drwxr-xr-x   - root supergroup          0 2020-10-11 21:11 actividad2
drwxr-xr-x   - root supergroup          0 2020-10-11 21:19 actividad3
drwxr-xr-x   - root supergroup          0 2020-10-11 20:56 userid-otros
drwxr-xr-x   - root supergroup          0 2020-10-11 20:55 userid-profile


In [17]:
!hdfs dfs -cat actividad3/part-r-00000

3	2
4	14
7	2
15	2
16	122
17	444
18	65
19	4514
20	5760
21	1589
22	7038
23	2552
24	1313
25	1289
26	2527
27	2360
28	1330
29	359
30	617
31	146
32	433
33	1441
34	404
35	1543
36	1962
38	422
39	213
42	13
48	35
75	109
103	1


Responda la siguiente pregunta:

¿ Cuántos usuarios de 35 años han escuchado al artista más popular ?